## Esercitazione Avanzata: Deployment Asincrono di un Modello di Object Detection

**Obiettivo dell'esercitazione:**
Superare i limiti di un'applicazione monolitica e sincrona. In questa esercitazione, progetterai e implementerai un'architettura a microservizi per il deployment di un modello di object detection (YOLOv8). Imparerai a gestire task di lunga durata in modo asincrono utilizzando una coda di messaggi e a orchestrare più container con Docker Compose.

**Architettura target:**
Costruirai un sistema composto da tre servizi containerizzati:
1. **Web API (Flask):** Riceve le richieste dall'utente, le accoda e restituisce un ID di tracciamento.
2. **ML Worker (Celery):** Esegue la pesante inferenza del modello in background, prelevando i task dalla coda.
3. **Message Broker (Redis):** Gestisce la comunicazione asincrona tra l'API e il Worker.

**Strumenti richiesti:**

  * Editor di codice (es. VSCode).
  * Python e `pip` installati localmente.
  * Docker Desktop con Docker Compose.

### Procedimento

#### Parte 1: Setup del Progetto e Logica Condivisa

1.  **Crea la struttura del progetto:** Organizza il tuo lavoro creando le seguenti cartelle nella root del tuo progetto: `api_service`, `ml_worker`, `shared`.
2.  **Configura Celery (`shared/celery_config.py`):**
      * Crea un file `celery_config.py` nella cartella `shared`.
      * In questo file, devi:
        1.  Inizializzare un'istanza di `Celery`, configurandola per usare Redis come `broker` e `backend`. L'URL di connessione per Docker Compose sarà `redis://redis:6379/0`.
        2.  Caricare un modello YOLOv8 pre-addestrato (es. `yolov8n.pt`) dalla libreria `ultralytics`. Questo caricamento deve avvenire a livello di modulo, in modo che sia eseguito una sola volta all'avvio del worker.
        3.  Definire un task Celery (es. `detect_objects`) che:
              * Accetti come input un'immagine codificata in stringa base64.
              * Decodifichi l'immagine in un formato utilizzabile da OpenCV/Numpy.
              * Esegua l'inferenza del modello sull'immagine.
              * Formatti i risultati (nomi delle classi, confidence, bounding box) in una stringa JSON e la restituisca.

#### Parte 2: Creazione dei Servizi Containerizzati

1.  **Sviluppa il servizio API (`api_service`):**

      * **`api.py`**: Crea un'API Flask con due endpoint:
          * `POST /predict`: Deve accettare un JSON con un'immagine codificata in base64. Invece di eseguire l'inferenza, deve chiamare il task Celery in modo asincrono (`task.delay(...)`) e restituire immediatamente un JSON con il `task_id` ricevuto.
          * `GET /result/<task_id>`: Deve usare l'ID per interrogare Celery sullo stato del task. Se il task è completato, restituisce i risultati; altrimenti, restituisce uno stato "PENDING".
      * **`requirements.txt`**: Elenca le dipendenze per l'API (Flask, Celery, Redis, Gunicorn).
      * **`Dockerfile`**: Scrivi un Dockerfile per containerizzare questo servizio. Ricorda di copiare sia il codice dell'API sia la cartella `shared` e di usare `gunicorn` come `CMD`.

2.  **Sviluppa il servizio ML Worker (`ml_worker`):**

      * **`worker.py`**: Questo script può essere molto semplice; il suo scopo è importare l'istanza di `celery_app` dalla configurazione condivisa in modo che il comando di avvio del worker possa trovarla.
      * **`requirements.txt`**: Elenca le dipendenze necessarie per l'inferenza (Celery, Redis, `ultralytics`, `torch`, `opencv-python-headless`, etc.). Queste saranno diverse e più pesanti di quelle dell'API.
      * **`Dockerfile`**: Scrivi un Dockerfile per questo servizio. Il `CMD` non sarà `gunicorn`, ma il comando per avviare un worker Celery, che deve puntare all'istanza dell'app Celery (es. `celery -A shared.celery_config.celery_app worker ...`).

#### Parte 3: Orchestrazione e Test Iniziale

1.  **Scrivi il file Docker Compose (`docker-compose.yml`):**

      * Nella root del progetto, crea un file `docker-compose.yml`.
      * Definisci tre `services`:
        1.  `redis`: Usa l'immagine ufficiale `redis:alpine`.
        2.  `api`: Configuralo per costruire l'immagine usando il `Dockerfile` in `api_service`. Mappa una porta del tuo PC (es. 5001) alla porta del container (5000) e definisci che `depends_on` `redis`.
        3.  `worker`: Configuralo per costruire l'immagine usando il `Dockerfile` in `ml_worker`. Questo servizio `depends_on` `redis` ma non ha bisogno di mappare porte.

2.  **Avvia e testa l'applicazione:**

      * Dal terminale, nella root del progetto, esegui `docker-compose up --build`. Osserva i log di tutti e tre i servizi che si avviano.
      * Prepara un'immagine di test e codificala in base64 per creare un file `payload.json`.
      * **Esegui il test a due fasi:**
        1.  Invia una richiesta `POST` a `http://localhost:5001/predict` con il tuo payload per avviare un task. Salva il `task_id` che ottieni in risposta.
        2.  Invia una o più richieste `GET` a `http://localhost:5001/result/<task_id>` fino a quando lo stato non diventa "COMPLETED" e vedi i risultati dell'object detection.

### Parte 4: Troubleshooting e Finalizzazione
Dopo aver implementato la versione iniziale, è molto probabile incontrare dei problemi. Questa è una parte normale del processo di sviluppo, specialmente in sistemi distribuiti. Di seguito sono elencati i problemi più comuni e le loro soluzioni, che renderanno la nostra applicazione più robusta e compatibile.

**Problema 1: Worker che crasha all'avvio (es. su Mac Apple Silicon)**
* **Sintomi**: Il worker fallisce con `exit code 132` o si blocca senza log.
* **Causa**: L'emulazione dell'architettura x86 su Apple Silicon può causare instabilità con librerie complesse come PyTorch/YOLO. Inoltre, caricare il modello a livello di modulo può essere problematico.
* **Soluzione**: Modificare `shared/celery_config.py` per un caricamento *lazy* del modello e, per garantire la compatibilità, usare un *modello mock* che simuli l'output di YOLO. Questo permette di testare l'intera architettura senza dipendere dall'hardware. Aggiungere anche `platform: linux/amd64` ai servizi `api` e `worker` nel `docker-compose.yml` per forzare l'emulazione corretta.

**Problema 2: Errore di importazione (`ModuleNotFoundError`)**
* **Sintomi**: L'API non si avvia e i log mostrano `ModuleNotFoundError: No module named 'cv2'` o `numpy`.
* **Causa**: Il file `celery_config.py` viene importato anche dall'API. Se questo file importa librerie (come `opencv` o `numpy`) che non sono state installate nel container dell'API, il servizio crasherà.
* **Soluzione**: Aggiungere `opencv-python-headless` e `numpy` al file `api_service/requirements.txt`.

**Problema 3: Conflitto di versione di NumPy**
* **Sintomi**: Errore `AttributeError: ARRAY_API not found` durante l'import di OpenCV.
* **Causa**: Incompatibilità tra le nuove versioni di NumPy (2.x) e la versione di OpenCV, che è stata compilata con una versione precedente (1.x).
* **Soluzione**: Specificare una versione compatibile di NumPy in **entrambi** i file `requirements.txt`, aggiungendo la riga: `numpy<2.0`.

**Problema 4: Dipendenze di sistema mancanti**
* **Sintomi**: Errori di importazione di OpenCV legati a file mancanti (es. `libGL.so.1`).
* **Causa**: L'immagine base di Python (`python:3.9-slim`) è minimale e non include le librerie di sistema necessarie per il rendering grafico usate da OpenCV.
* **Soluzione**: Modificare **entrambi** i `Dockerfile` (`api_service` e `ml_worker`) per installare queste dipendenze con `apt-get`:
    ```Dockerfile
    RUN apt-get update && apt-get install -y libgl1-mesa-glx libglib2.0-0 \
        && rm -rf /var/lib/apt/lists/*
    ```

### Parte 5: Test del Sistema Finalizzato
Dopo aver applicato tutte le correzioni, la tua architettura è robusta e pronta per un test completo.

1.  **Ricostruisci e avvia i container:**
    ```bash
    # Ferma e rimuovi eventuali container vecchi
    docker-compose down

    # Ricostruisci le immagini e avvia i servizi in background
    docker-compose up --build -d
    ```
2.  **Verifica che tutti i servizi siano attivi:**
    ```bash
    docker-compose ps
    ```
    Dovresti vedere i tre servizi (`api`, `worker`, `redis`) con lo stato `Up` o `running`.

3.  **Esegui nuovamente il test a due fasi:**
    * **A. Invia la richiesta:**
      ```bash
      curl -X POST "http://localhost:5001/predict" -H "Content-Type: application/json" -d @payload.json
      ```
      Riceverai come prima un `task_id`.

    * **B. Controlla il risultato:**
      ```bash
      curl http://localhost:5001/result/<IL_TUO_TASK_ID>
      ```
      Dopo pochi secondi, dovresti ricevere una risposta `COMPLETED` con i dati del modello mock, a dimostrazione che l'intero flusso asincrono funziona correttamente.

**Conclusione:**
Completando questa esercitazione, hai costruito e, soprattutto, **reso robusta** un'applicazione MLOps scalabile. Hai imparato a diagnosticare e risolvere problemi comuni legati alle dipendenze, alla compatibilità tra architetture e all'orchestrazione di container, acquisendo competenze fondamentali per il deployment di sistemi di machine learning nel mondo reale.